# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

In [ ]:
#import pandas as pd
#import numpy as np

#https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/automated-machine-learning/classification-bank-marketing-all-features/auto-ml-classification-bank-marketing-all-features.ipynb

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

In the Automated ML run a classification task ............ is done using the .........dataset.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# choose a name for experiment
experiment_name = 'capstone-automl-exp'
project_folder = './pipeline-project'

experiment=Experiment(ws, experiment_name)
experiment

Create compute cluster

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

amlcompute_cluster_name = "cluster-automl"

try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 5) #timeout_in_minutes = 10

Embed the dataset

In [ ]:
found = False
key = "heard-db" #rename->chose dataset
description_text = "Kaggle heard data database" #rename

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
        #print("found!")

if not found: 
        # Create AML Dataset and register it into Workspace
        #print("not found!") #
        example_data = 'https://github.com/Daniel-car1/nd00333-capstone/blob/main/heart.csv'  
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

In [ ]:
dataset.take(5).to_pandas_dataframe()

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="target",    #anpassen
                             path = project_folder,
                             enable_onnx_compatible_models=True,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output= True)

In [ ]:
best_run_customized, fitted_model_customized = remote_run.get_output()

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Wait for the best model explanation run to complete
from azureml.core.run import Run
model_explainability_run_id = remote_run.id + "_" + "ModelExplain"
print(model_explainability_run_id)
model_explainability_run = Run(experiment=experiment, run_id=model_explainability_run_id)
model_explainability_run.wait_for_completion()

# Get the best run object
#best_run, fitted_model = remote_run.get_output() #  <--- vielleicht braucht man das doch??

In [ ]:
# SAVE BEST RUN

In [ ]:
best_run, onnx_mdl = remote_run.get_output(return_onnx_model=True)

In [ ]:
from azureml.automl.runtime.onnx_convert import OnnxConverter
onnx_fl_path = "./best_model.onnx"
OnnxConverter.save_onnx_model(onnx_mdl, onnx_fl_path)

In [ ]:
# Get the best run object
best_run, fitted_model = remote_run.get_output()

In [ ]:
model_name = best_run.properties['model_name']
print(best_run)

In [ ]:
print(fitted_model[-1])

In [ ]:
best_run.get_metrics()

In [ ]:
script_file_name = 'inference/score.py'

best_run.download_file('outputs/scoring_file_v_1_0.py', 'inference/score.py')

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
#Register the Fitted Model for Deployment
model_name = best_run.properties['model_name']
description = 'AutoML Model trained on heart data'
tags = None
model = remote_run.register_model(model_name = model_name, description = description, tags = tags)

print(remote_run.model_id) # This will be written to the script file later in the notebook.  

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
inference_config = InferenceConfig(entry_script=script_file_name, environment=myenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 2, 
                                               enable_app_insights=True, 
                                               auth_enabled=True,
                                               tags = {'area': "haData", 'type': "automl_classification"},
                                               enable_app_insights=True,
                                               description = 'service for Automl Classification')

aci_service_name = 'automl-heart-endpoint'
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig) #,overwrite=True
aci_service.wait_for_deployment(show_output=True)
print(aci_service.state)

In [ ]:
#logging
from azureml.core import Workspace
from azureml.core.webservice import Webservice
#ws = Workspace.from_config()
service = Webservice(name=aci_service_name, workspace=ws)
logs = service.get_logs()

for line in logs.split('\n'):
    print(line)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
#den absatz vielleicht wegwerfen?!

#import json
#import requests

#X_test_json = X_test.to_json(orient='records')
#data = "{\"data\": " + X_test_json +"}"
#headers = {'Content-Type': 'application/json'}

#resp = requests.post(aci_service.scoring_uri, data, headers=headers)

#y_pred = json.loads(json.loads(resp.text))['result']

In [ ]:
import json
import requests

scoring_uri = aci_service.scoring_uri() 
#key=''
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'
data = {"data":
        [
          {
            "age":37,
            "sex": 1,
            "cp": 2,
            "trestbps": 130,
            "chol": 250,
            "fbs": 0,
            "restecg": 1,
            "thalach": 187,
            "exang": 0,
            "oldpeak": 3.5,
            "slope": 0,
            "ca": 0,
            "thal": 2,
          }
      ]
    }
test_sample = json.dumps(data)


resp = requests.post(scoring_uri, test_sample, headers=headers)


In [ ]:
print("prediction " , resp.json())

In [ ]:
y_pred = json.loads(json.loads(resp.text))['result']
print("prediction ", y_pred)

In [ ]:
print("expected result: 1")

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
logs = aci_service.get_logs()
for line in logs.split('\n'):
    print(line)

In [ ]:
aci_service.delete()

In [ ]:
compute_target.delete()

In [ ]:
print('finished')